In [1]:
from src.SIV_library.lib import OfflinePIV
import torch

import numpy as np
import matplotlib.pyplot as plt

import cv2
import os
import gc

KeyboardInterrupt: 

In [2]:
gpu_enabled = torch.cuda.is_available()
# gpu_enabled = False

if gpu_enabled:
    device = torch.cuda.get_device_name()
else:
    device = 'cpu'
    
print("Current device:", device)

Current device: cpu


In [4]:
for f in os.listdir('test'):
    os.remove(f'test/{f}')

test_arr = np.random.choice([0, 255], size=(128, 128), p=[19/20, 1/20]).astype(np.uint8)
cv2.imshow('', test_arr)
cv2.waitKey(1000)
cv2.destroyAllWindows()

for i in range(50):
    cv2.imwrite(f'test/{i}.jpg', test_arr)
    test_arr = np.roll(test_arr, 3)

In [5]:
# folder, file_format, (mode, dt) = r"test images", "bmp", ("pairs", 12)
# folder, file_format, (mode, dt) = r"plume simulation", "jpg", ("sequential", 500)
folder, file_format, (mode, dt) = r"test", "jpg", ("sequential", 500)

In [5]:
torch.cuda.empty_cache()
gc.collect()

886

In [8]:
piv_gen = OfflinePIV(
    folder=folder, # Path to experiment
    device=device, # Device name
    file_fmt=file_format,
    wind_size=32,
    overlap=16,
    dt=dt, # Time between frames, mcs
    scale = 0.02, # mm/pix
    multipass=2,
    multipass_mode="CWS", # CWS or DWS
    multipass_scale=2.0, # Window downscale on each pass
    folder_mode=mode # Pairs or sequential frames 
)

results = []
for out in piv_gen():
    x, y, vx, vy = out
    results.append(out)

Load time 0.007 sec Correlating... torch.Size([49, 32, 32]) torch.Size([49, 32, 32])
Correlating... torch.Size([225, 16, 16]) torch.Size([225, 16, 16])
Iteration finished in 0.023 sec Warning! to many false vectors
Warning! to many false vectors
Batch finished in 0.035 sec
Load time 0.002 sec Correlating... torch.Size([49, 32, 32]) torch.Size([49, 32, 32])
Correlating... torch.Size([225, 16, 16]) torch.Size([225, 16, 16])
Iteration finished in 0.021 sec Warning! to many false vectors
Warning! to many false vectors
Batch finished in 0.027 sec
Load time 0.002 sec Correlating... torch.Size([49, 32, 32]) torch.Size([49, 32, 32])
Correlating... torch.Size([225, 16, 16]) torch.Size([225, 16, 16])
Iteration finished in 0.021 sec Warning! to many false vectors
Warning! to many false vectors
Batch finished in 0.029 sec
Load time 0.002 sec Correlating... torch.Size([49, 32, 32]) torch.Size([49, 32, 32])
Correlating... torch.Size([225, 16, 16]) torch.Size([225, 16, 16])
Iteration finished in 0.01

In [7]:
files = os.listdir(folder)

for idx, (x, y, vx, vy) in enumerate(results):
    img = cv2.imread(folder + f"/{files[idx]}", cv2.IMREAD_GRAYSCALE)

    new_x = x/piv_gen._scale
    new_y = y/piv_gen._scale

    plt.imshow(img, cmap='gray')
    plt.quiver(new_x, new_y, vx, vy, color='red')
    plt.show()

KeyboardInterrupt: 

In [6]:
for img_file in os.listdir(folder):
    img = cv2.imread(folder + f"/{img_file}", cv2.IMREAD_GRAYSCALE)
    xsize, ysize = img.shape
    
    newx, newy = int(xsize), int(ysize)
    img = cv2.resize(img, (newx, newy), interpolation= cv2.INTER_LINEAR)
    
    cv2.imshow(str(img_file), img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()